# Modele scoringowe

Kolokwium by kkedzierski

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, auc
import scorecardpy as sc

## Notatki

### Szansa (odds)

Szansa - inny sposób wyrażania prawdopodobnieństwa (częstości)

$O(B)$ - szansa złego:

$$
O(B) = P(B) / P(G) = \frac{P(B)}{1 - P(B)}
$$

$O(G)$ - szansa dobrego:

$$
O(G) = \frac{P(G)}{1 - P(G)}
$$

### Logarytm szansy

$L(B)$ - logarytm szansy złego:

$$
L(B) = ln(O(B)) = ln(\frac{P(B)}{P(G)})
$$

$L(G)$ - logarytm szansy dobrego:

$$
L(G) = ln(O(G)) = ln(\frac{P(G)}{P(B)})
$$

### Wzór Bayesa

$$
P(A \mid B) = \frac{P(B \mid A) \cdot P(A)}{P(B)}
$$

Jeszcze przydatny jest wzór na prawdopodobieństwo całkowite:

$$
P(B) = \sum_{i=1}^n P(B \mid A_i) \cdot P(A_i)
$$

#### Zadanie z wykładu - rzadka choroba

Zalózmy, że mamy do czynienia z wzglednie rzadką chorobą. W populacji choruje na nią jedna na 1000 osób. Mamy tez niedoskonłay test, którego zadaniem jest wykryć tę chorobę. Jego czułość (prawdopodobieństwo, ze zwróci on wynik pozytywny - to znaczy wykryje chorobe - u osoby, która jest chora) to 0,95. Prawdopodobienstwo błędnego („fałszywego") pozytywnego wyniku u zdrowej osoby wynosi 0,01.

Z populacji losujemy osobe i przeprowadzamy omawiany test. Zwraca on wynik pozytywny. Jakie jest prawdopodobienstwo, ze ta osoba jest chora?

In [8]:
hypotheses = ["chory", "nie chory"]
prior = np.array([0.001, 0.999])
likelihood = np.array([0.95, 0.01])

posterior = prior * likelihood / sum(prior * likelihood)

posterior

array([0.08683729, 0.91316271])

...wynosi ok. 0.0868

#### Zadanie z wykładu - Alejandro i Beatriz

Janusz dość często podróżuje do Alicante, gdzie mieszkają jego przyjaciele Alejandro i Beatriz. Jedno z nich zawsze odbiera Janusza z lotniska. Załóżmy, ze oboje równie często przyjeżdzają po Janusza. Mają oni dwa auta: czerwone i srebrne. Beatriz woli czerwone auto; korzysta z czerwonego auta w 60% przypadków. Alejandro używa srebrnego auta 4 razy częściej niż czerwonego. Alejandro częściej się spóźnia. Jeżeli Janusza odbiera Alejandro, prawdopodobieństwo, że będzie na czas, wynosi 0,4; Beatriz zdąży na czas z prawdopodobieństwem 0,8. Beatriz lubi używać klaksonu. Jeżeli zobaczy Janusza, zatrąbi w ośmiu przypadkach na 10. Alejandro tez czasem używa klaksonu - prawdopodobieństwo, ze zatrąbi, widząc Janusza, wynosi 0,2. Janusz właśnie wyszedł z hali przylotów i widzi, ze czeka już na niego srebrne auto, a jego kierowca daje mu sygnał klaksonem. Jakie jest prawdopodobieństwo, ze to Beatriz? Załóżmy, ze używanie klaksonu, kolor auta i przybycie na czas bądź spóznienie się są niezależne od siebie

#### Rozwiązanie po kolei

In [ ]:
# Dane: Samochód o kolorze srebrnym
hypotheses = ["Alejandro", "Beatriz"]

prior = np.array(
    [0.5, 0.5]
)  # Prawdopodobieństwo, że przyjedzie Alejandro i że przyjedzie Beatriz
likelihood = np.array(
    [0.8, 0.4]
)  # Prawdopodobieństwo wyboru koloru srebrnego przez Alejandro i Beatriz

posterior = prior * likelihood / sum(prior * likelihood)
posterior
# Posterior: Prawdopodobieśtwo, że przyjechał srebrny samochód pod warunkiem, że przyjechał Alejandro lub Beatriz

array([0.66666667, 0.33333333])

In [ ]:
# Dane: Kierowca jest na czas

# Posterior staje się priorem
prior = posterior
likelihood = np.array(
    [0.4, 0.8]
)  # Prawdopodobieństwo, że Alejandro i Beatriz przyjadą na czas

posterior = prior * likelihood / sum(prior * likelihood)
posterior
# Posterior: Prawdopodobieństwo, że przyjechał Alejandro lub Beatriz pod warunkiem, że przyjechali na czas i że samochód był srebrny

array([0.5, 0.5])

In [ ]:
# Dane: Kierowca zatrąbił

# Posterior staje się priorem
prior = posterior
likelihood = np.array([0.2, 0.8])  # Prawdopodobieństwo, że Alejandro i Beatriz zatrąbią
posterior = prior * likelihood / sum(prior * likelihood)
posterior
# Posterior: Prawdopodobieństwo, że przyjechał Alejandro lub Beatriz pod warunkiem, że zatrąbili, że samochód był srebrny i że przyjechali na czas

array([0.2, 0.8])

...prawdopodobieństwo, że to Beatriz wynosi 0.8.

#### Rozwiązanie naraz

In [ ]:
hypotheses = ["Alejandro", "Beatriz"]
prior = np.array([0.5, 0.5])
l_silver = np.array(
    [0.8, 0.4]
)  # Prawdopodobieństwo wyboru koloru srebrnego przez Alejandro i Beatriz
l_time = np.array(
    [0.4, 0.8]
)  # Prawdopodobieństwo, że Alejandro i Beatriz przyjadą na czas
l_horn = np.array([0.2, 0.8])  # Prawdopodobieństwo, że Alejandro i Beatriz zatrąbią

posterior = prior * l_silver * l_time * l_horn / sum(prior * l_silver * l_time * l_horn)

posterior


array([0.2, 0.8])

#### Rozwiązanie "na kartce"

#### Oznaczenia i dane 

$$
\begin{aligned}
B &: \text{Beatriz przyjeżdża} & P(B)=\tfrac12 \\[4pt]
A &: \text{Alejandro przyjeżdża} & P(A)=\tfrac12
\end{aligned}
$$

| Zmienna | Znaczenie | Beatriz | Alejandro |
|---------|-----------|---------|-----------|
| $S$ | auto jest **srebrne** | $P(S\mid B)=0{,}4$ | $P(S\mid A)=0{,}8$ |
| $H$ | kierowca używa **klaksonu** | $P(H\mid B)=0{,}8$ | $P(H\mid A)=0{,}2$ |
| $T$ | kierowca jest **na czas** | $P(T\mid B)=0{,}8$ | $P(T\mid A)=0{,}4$ |

Dla każdego kierowcy zdarzenia $S, H, T$ są **wzajemnie niezależne**.

#### Prawdopodobieństwo jednoczesnego wystąpienia $S \cap H \cap T$

$$
\begin{aligned}
P(S \cap H \cap T \mid B) &= P(S\mid B)\,P(H\mid B)\,P(T\mid B) \\
                          &= 0{,}4 \times 0{,}8 \times 0{,}8 \\
                          &= 0{,}256, \\[8pt]
P(S \cap H \cap T \mid A) &= P(S\mid A)\,P(H\mid A)\,P(T\mid A) \\
                          &= 0{,}8 \times 0{,}2 \times 0{,}4 \\
                          &= 0{,}064.
\end{aligned}
$$

#### Zastosowanie wzoru Bayesa

$$
\begin{aligned}
P(B \mid S, H, T)
&= \frac{P(S, H, T \mid B)\,P(B)}
        {P(S, H, T \mid B)\,P(B) + P(S, H, T \mid A)\,P(A)} \\[10pt]
&= \frac{0{,}256 \times 0{,}5}
        {0{,}256 \times 0{,}5 + 0{,}064 \times 0{,}5} \\[10pt]
&= \frac{0{,}128}{0{,}128 + 0{,}032} \\[4pt]
&= \frac{0{,}128}{0{,}160} \\[4pt]
&= 0{,}80.
\end{aligned}
$$

### Odpowiedź  

$$
{P(B \mid \text{srebrne auto},\,\text{klakson},\,\text{na czas}) = 0{,}80 = 80\%}
$$


No i elegancko

### Tablica pomyłek

| Klasa rzeczywista ↓ \ Klasa predykowana → | Klasyfikacja pozytywna           | Klasyfikacja negatywna             |
|--------------------------------------|----------------------------------|------------------------------------|
| **Stan pozytywny**                   | **Prawdziwie dodatnia (TP)**     | **Fałszywie ujemna (FN)**          |
| **Stan negatywny**                   | **Fałszywie dodatnia (FP)**      | **Prawdziwie ujemna (TN)**         |

#### Ocena jakości modelu

| Miara                        | Wzór                                      | Opis                                                |
|------------------------------|-------------------------------------------|-----------------------------------------------------|
| **Dokładność** (accuracy)    | $$ \frac{TP + TN}{TP + TN + FP + FN} $$   | Udział poprawnych predykcji                        |
| **Precyzja** (precision)     | $$ \frac{TP}{TP + FP} $$                  | Udział trafień wśród pozytywnych predykcji         |
| **Czułość** (recall)         | $$ \frac{TP}{TP + FN} $$                  | Udział trafień wśród rzeczywistych pozytywnych     |
| **Specyficzność**            | $$ \frac{TN}{TN + FP} $$                  | Udział trafień wśród rzeczywistych negatywnych     |
| **F1-score**                 | $$ \frac{2 \cdot TP}{2 \cdot TP + FP + FN} $$ | Harmoniczna średnia precyzji i czułości        |

## Zadania obliczeniowe

### Zadanie 1

*Mamy dane:*

`dane.csv` (`data/dane.csv`)

*I następującą tabelę scoringową:*

| Zmienna                                      | Kategoria   | Punkty |
|---------------------------------------------|-------------|--------|
| Liczba opóźnień w ostatnich 12 miesiącach    | 0           | 25     |
|                                             | 1           | 12     |
|                                             | 2+          | 0      |
| Wykorzystanie limitu odnawialnego w %       | (0;16]      | 30     |
|                                             | (16;28]     | 22     |
|                                             | (28;40]     | 15     |
|                                             | >40         | 0      |

1. *Ile wynosi średnia ocena punktowa (dwie cyfry po przecinku)?*
2. *Ile wynosi minimalna ocena punktowa?*
3. *Ile wynosi maksymalna ocena punktowa?*
4. *Ile wynosi mediana ocen punktowych?*
5. *Ile wynosi AUC (trzy cyfry po przecinku)?*
6. *Ile wynosi Gini (trzy cyfry po przecinku)?*

#### Rozwiązanie

Ładowanie danych

In [41]:
df = pd.read_csv("./data/dane-3.csv", sep=";")

# Convert 'wykorz_limit' from string to float
df["wykorz_limit"] = df["wykorz_limit"].str.replace(",", ".").astype(float)

df.head()

,liczba_opozn,wykorz_limit,bad_flag
0,1,0.30,0
1,2,0.69,1
2,0,0.66,0
3,0,0.39,0
4,1,0.49,1


Funkcja scoringowa

In [69]:
def calculate_score(row: pd.Series) -> int:
    # Base score
    score: int = 0

    # Score for `liczba_opozn` column
    if row["liczba_opozn"] == 0:
        score += 28
    elif row["liczba_opozn"] == 1:
        score += 15
    elif row["liczba_opozn"] >= 2:
        score += 0

    # Score for `wykorz_limit` colun
    if 0.0 < row["wykorz_limit"] <= 0.15:
        score += 32
    elif 0.15 < row["wykorz_limit"] <= 0.26:
        score += 25
    elif 0.26 < row["wykorz_limit"] <= 0.38:
        score += 17
    elif row["wykorz_limit"] > 0.38:
        score += 0

    # Return calculated score
    return score

Aplikowanie scoringu

In [70]:
df["score"] = df.apply(calculate_score, axis=1)

df.head()

,liczba_opozn,wykorz_limit,bad_flag,score
0,1,0.30,0,32
1,2,0.69,1,0
2,0,0.66,0,28
3,0,0.39,0,28
4,1,0.49,1,15


1. *Ile wynosi średnia ocena punktowa (dwie cyfry po przecinku)?*

In [71]:
print(df["score"].mean())

38.7065


2. *Ile wynosi minimalna ocena punktowa?*

In [72]:
print(df["score"].min())

0


3. *Ile wynosi maksymalna ocena punktowa?*

In [73]:
print(df["score"].max())

60


4. *Ile wynosi mediana ocen punktowych?*

In [74]:
print(df["score"].median())

45.0


5. *Ile wynosi AUC (trzy cyfry po przecinku)?*

**Uwaga**: `score` jest odwrócony, ponieważ w problemie scoringowym zachodzi odwrotna zależność między uzyskanym `score` a prawdopodobieństwem przynależności do `bad_flag`

In [75]:
# Invert the score to a negative value for proper scoring problem ROC AUC calculation
auc_ = roc_auc_score(df["bad_flag"], -df["score"])

print(auc_)

0.7515919171387524


6. *Ile wynosi Gini (trzy cyfry po przecinku)?*

In [76]:
gini = 2 * auc_ - 1

print(gini)

0.5031838342775048


In [77]:
df2 = pd.read_csv("./data/dane-3-rozwiazanie.csv", sep=";")

df2.head()

,liczba_opozn,wykorz_limit,bad_flag,points1,points2,score
0,1,"0,3",0,15,17,32
1,2,"0,69",1,0,0,0
2,0,"0,66",0,28,0,28
3,0,"0,39",0,28,0,28
4,1,"0,49",1,15,0,15


In [78]:
# Identify rows where the 'score' column differs between df and df2
mismatch_mask = df["score"] != df2["score"]

if mismatch_mask.any():
    # Select the mismatching rows from df
    df_mismatched = df.loc[mismatch_mask, ["liczba_opozn", "wykorz_limit", "score"]]

    # Select the mismatching rows from df2, including 'points1' and 'points2' for context
    df2_mismatched = df2.loc[
        mismatch_mask, ["liczba_opozn", "wykorz_limit", "points1", "points2", "score"]
    ]

    # Concatenate the selected columns from both dataframes side-by-side for comparison
    # Add suffixes to column names to indicate their original dataframe
    comparison_df = pd.concat(
        [df_mismatched.add_suffix("_df"), df2_mismatched.add_suffix("_df2")], axis=1
    )

    print("Mismatching rows (score column wise):")
else:
    print("No mismatching scores found between df and df2.")


Mismatching rows (score column wise):


In [79]:
comparison_df

,liczba_opozn_df,wykorz_limit_df,score_df,liczba_opozn_df2,wykorz_limit_df2,points1_df2,points2_df2,score_df2
194,0,0.0,28,0,0,28,32,60
581,0,0.0,28,0,0,28,32,60
659,0,0.0,28,0,0,28,32,60
1006,1,0.0,15,1,0,15,32,47
1561,0,0.0,28,0,0,28,32,60
1659,0,0.0,28,0,0,28,32,60
1756,0,0.0,28,0,0,28,32,60
1953,0,0.0,28,0,0,28,32,60


### Zadanie 2

*Krzywa ROC wygląda następująco:*

![](./static/wykresROC-1.png)

*Współrzędne linii ROC:*

```r
x <- c(0, 0.16, 0.23, 0.47, 1)
y <- c(0, 0.68, 0.83, 0.92, 1)
```

*Oblicz:*

1. *AUC=*
2. *Gini=*

#### Rozwiązanie

Obliczanie AUC

In [40]:
x = [0, 0.16, 0.23, 0.47, 1]
y = [0, 0.68, 0.83, 0.92, 1]

auc_ = auc(x, y)

1. *AUC=*

In [41]:
print(auc_)

0.82605


2. *Gini=*

In [42]:
gini = 2 * auc_ - 1

print(gini)

0.6520999999999999


### Zadanie 3

*Częstość złych (bad rate) wynosi 36%*

*Uwaga! W odpowiedziach należy podawać przynajmniej 3 cyfry po przecinku.*

1. *Szansa dobrego wynosi:*
2. *Logarytm szansy dobrego wynosi:*
3. *Logarytm szansy złego wynosi:*

#### Rozwiązanie

In [49]:
bad_rate = 0.36

1. *Szansa dobrego wynosi:*

In [50]:
good_odds = (1 - bad_rate) / bad_rate

print(good_odds)

1.777777777777778


2. *Logarytm szansy dobrego wynosi:*


In [53]:
good_log_odds = np.log(good_odds)

print(good_log_odds)

0.5753641449035619


3. *Logarytm szansy złego wynosi:*

In [58]:
# Solution 1
bad_odds = bad_rate / (1 - bad_rate)

bad_log_odds = np.log(bad_odds)

print(bad_log_odds)

-0.5753641449035618


In [59]:
# Solution 2
bad_log_odds = -good_log_odds

print(bad_log_odds)

-0.5753641449035619


### Zadanie 4

*Logarytm szansy dobrego wynosi 4.*

*Uwaga! W odpowiedziach należy podawać przynajmniej 3 cyfry po przecinku.*

1. *Prawdopodobieństwo złego wynosi:*
2. *Szansa złego wynosi:*
3. *Szansa dobrego wynosi:*

#### Rozwiązanie

In [83]:
good_log_odds = 4

1. *Prawdopodobieństwo złego wynosi:*

In [84]:
prob_good = np.exp(good_log_odds) / (1 + np.exp(good_log_odds))

prob_bad = 1 - prob_good

print(prob_bad)

0.01798620996209155


2. *Szansa złego wynosi:*

In [85]:
bad_odds = prob_bad / prob_good

print(bad_odds)

0.01831563888873417


3. *Szansa dobrego wynosi:*

In [86]:
good_odds = prob_good / prob_bad

print(good_odds)

54.59815003314427


### Zadanie 5

*Mamy dane:*

`dane.csv` (`./data/dane-2.csv`)

*W odpowiedziach należy podać przynajmniej trzy miejsca po przecinku.*

1. *Ile wynosi IV zmiennej grupa_wiekowa?*
2. *Ile wynosi IV zmiennej grupa_dochodowa?*

#### Rozwiązanie

In [89]:
df = pd.read_csv("./data/dane-2.csv", sep=";")

df.head()

,grupa_wiekowa,grupa_dochodowa,bad
0,"(51,67]",[1000-2000),1
1,"(34,51]",[2000-3000),0
2,"(51,67]",[4000-5000),1
3,"(51,67]",[4000-5000),0
4,"(34,51]",[1000-2000),0


1. *Ile wynosi IV zmiennej grupa_wiekowa?*
2. *Ile wynosi IV zmiennej grupa_dochodowa?*

In [93]:
sc.iv(df, y="bad")

,variable,info_value
0,grupa_wiekowa,0.213633
1,grupa_dochodowa,0.009519


### Zadanie 6

*W pewnym południowoamerykańskim państwie jest 100 razy więcej rolników niż bibliotekarzy. 5% rolników to ludzie nieśmiali, zaś wśród bibliotekarzy udział osób nieśmiałych stanowi 76%*

1. *Losujemy jedną osobę z listy zawierającej wszystkich rolników i bibliotekarzy. Jakie jest prawdopodobieństwo, że jest nieśmiała?*
2. *Losujemy jedną osobę z listy zawierającej wszystkich rolników i bibliotekarzy. Stwierdzamy, że jest nieśmiała. Jakie jest prawdopodobieństwo, że jest bibliotekarzem?*

#### Rozwiązanie

#### Oznaczenia  

$$
\begin{aligned}
F &: \text{rolnik}, & P(F)=\frac{100}{101} \\[4pt]
B &: \text{bibliotekarz}, & P(B)=\frac{1}{101} \\[8pt]
S &: \text{osoba nieśmiała}, &
\begin{cases}
P(S\mid F)=0{,}05,\\[2pt]
P(S\mid B)=0{,}76
\end{cases}
\end{aligned}
$$

#### Prawdopodobieństwo, że losowo wybrana osoba jest nieśmiała

Z całkowitego prawdopodobiweństwa:

$$
\begin{aligned}
P(S)
&= P(S\mid F)\,P(F) + P(S\mid B)\,P(B) \\[6pt]
&= 0{,}05\cdot\frac{100}{101} + 0{,}76\cdot\frac{1}{101} \\[6pt]
&= \frac{5}{101} + \frac{0{,}76}{101} \\[2pt]
&= \frac{5{,}76}{101} \approx 0{,}0570.
\end{aligned}
$$

#### 2. Prawdopodobieństwo, że nieśmiała osoba jest bibliotekarzem  

Z Bayesa:

$$
\begin{aligned}
P(B \mid S)
&= \frac{P(S\mid B)\,P(B)}{P(S)} \\[6pt]
&= \frac{0{,}76\cdot\frac{1}{101}}{\frac{5{,}76}{101}} \\[6pt]
&= \frac{0{,}76}{5{,}76}
= \frac{76}{576}
= \frac{19}{144} \approx 0{,}1319.
\end{aligned}
$$


1. *Losujemy jedną osobę z listy zawierającej wszystkich rolników i bibliotekarzy. Jakie jest prawdopodobieństwo, że jest nieśmiała?*

In [ ]:
p_n = 0.05 * (100 / 101) + 0.76 * (1 / 101)

print(p_n)

0.05702970297029703


2. *Losujemy jedną osobę z listy zawierającej wszystkich rolników i bibliotekarzy. Stwierdzamy, że jest nieśmiała. Jakie jest prawdopodobieństwo, że jest bibliotekarzem?*

In [ ]:
0.76 * (1 / 101) / p_n

0.13194444444444445

### Pozostałe

Jakieś rzeczy